In [45]:
import IMP
import IMP.atom
import IMP.algebra
import os
import glob
from pathlib import Path

# Set up paths
akt_pdb = "../data/akt_136.pdb"
input_dir = "../data/pdbs/"
output_dir = "../data/pdbs_align/"

# Create output directory if it doesn't exist
Path(output_dir).mkdir(parents=True, exist_ok=True)


In [46]:
# Load reference structure
m_akt = IMP.Model()
h_akt = IMP.atom.read_pdb(akt_pdb, m_akt)

# Get all atoms from reference structure for alignment
pids_akt = IMP.atom.Selection(h_akt).get_selected_particles()


begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: CA " in residue #129 "BEA"
end read_pdb


In [50]:
def align_pdb_to_reference(h_target, h_ref):
    """
    Align a target hierarchy to the reference hierarchy in place.

    Parameters:
    -----------
    target_hierarchy : IMP.atom.Hierarchy
        Target structure hierarchy to be aligned (modified in place)
    reference_hierarchy : IMP.atom.Hierarchy
        Reference structure hierarchy
    """
    pids_ref = list()
    pids_target = list()
    for residue_index in [353, 241, 263]:
        pids_ref.append(IMP.atom.Selection(h_ref, residue_index=residue_index, atom_type=IMP.atom.AT_CA).get_selected_particles()[0])
        pids_target.append(IMP.atom.Selection(h_target, residue_index=residue_index,atom_type=IMP.atom.AT_CA).get_selected_particles()[0])

    print(len(pids_ref), len(pids_target))

    m_ref = h_ref.get_model()
    m_target = h_target.get_model()

    coords_ref = [IMP.core.XYZ(m_ref, pid).get_coordinates() for pid in pids_ref]
    coords_target = [IMP.core.XYZ(m_target, pid).get_coordinates() for pid in pids_target]

    # Compute transformation to align target to reference
    transformation = IMP.algebra.get_transformation_aligning_first_to_second(
        coords_target, coords_ref
    )

    # # Apply transformation to all particles in target
    ps = IMP.atom.get_leaves(h_target)
    for p in ps:
        if IMP.core.XYZ.get_is_setup(p):
            xyz = IMP.core.XYZ(p)
            xyz.set_coordinates(transformation.get_transformed(xyz.get_coordinates()))


In [51]:
# Get all PDB files in the input directory
pdb_files = sorted(glob.glob(os.path.join(input_dir, "*.pdb")))
print(f"Found {len(pdb_files)} PDB files to align")


Found 63 PDB files to align


In [52]:
# Process all PDB files
successful = 0
failed = 0

## 353 vs 208
for i, pdb_file in enumerate(pdb_files):
    print(pdb_file)
    m_ref = IMP.Model()
    h_ref = IMP.atom.read_pdb(pdb_file, m_akt)

    align_pdb_to_reference(h_akt, h_ref)

    # # Create output filename and save
    basename = os.path.basename(pdb_file)
    output_file = os.path.join(output_dir, basename)
    IMP.atom.write_pdb(h_akt, output_file)

../data/pdbs/0.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/1.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/10.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/11.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/12.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/13.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/14.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/15.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/16.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pdbs/17.pdb
3 3
../data/pdbs/18.pdb
WARNING  Degenerate point set. I may not be able to align them.
3 3
../data/pdbs/19.pdb
3 3
WARNING  Degenerate point set. I may not be able to align them.
../data/pd